In [1]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown
from src.dbhandler import DBHandler 

In [2]:
def initialize_dropdowns():
    """Initialize study and board size dropdowns based on available data."""
    current_studies_dict = db_handler.get_study_names()
    initial_study_id = list(current_studies_dict.values())[0]

    # Create study dropdown
    study_dropdown = widgets.Dropdown(
        options=current_studies_dict,
        description="Study ID:",
        value=initial_study_id
    )

    # Set board sizes based on initial study selection
    board_size_dropdown = widgets.Dropdown(
        options=db_handler.load_board_sizes(initial_study_id),
        description="Board Size:",
        value=6
    )

    # Update board size dropdown options when study changes
    def on_study_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            board_size_dropdown.options = db_handler.load_board_sizes(change['new'])
    
    study_dropdown.observe(on_study_change)
    
    return study_dropdown, board_size_dropdown

In [3]:
def format_dataframe(df):
    """Format integer and float columns."""
    int_columns = ["number_of_clauses", "board_size", "depth", "epochs", "mbf_idx", "n_samples_idx", "open_pos"]
    float_columns = ["f1", "precision", "recall", "acc_train", "acc_test"]

    for col in int_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int).apply(lambda x: f"{x:,}")
    for col in float_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').apply(lambda x: f"{x:.6f}" if pd.notna(x) else "")

    return df

In [4]:
def highlight_rows(s):
    """Highlight rows based on acc_test values."""
    return [
        'background-color: green' if v == 100.0 else 
        'background-color: yellow' if v >= 99.0 else
        'background-color: lightyellow' if v >= 95.0 else
        'background-color: orange' if v >= 90.0 else
        'background-color: red' if v < 90.0 else
        '' for v in s
    ]

In [5]:
def query_top_results(study_id, board_size, sort_order, max_rows):
    results_df = db_handler.query_top_results(study_id, board_size)

    # Pivot parameters
    if 'paramname' in results_df.columns and 'paramvalue' in results_df.columns:
        param_pivot = results_df.pivot_table(
            index=['number', 'state', 'value'],
            columns='paramname',
            values='paramvalue',
            aggfunc='first'
        ).reset_index()
    else:
        param_pivot = pd.DataFrame()  # Empty if columns are missing
    
    # Pivot user attributes
    if 'userattributekey' in results_df.columns and 'userattributevalue' in results_df.columns:
        attr_pivot = results_df.pivot_table(
            index=['number', 'state', 'value'],
            columns='userattributekey',
            values='userattributevalue',
            aggfunc='first'
        ).reset_index()
    else:
        attr_pivot = pd.DataFrame()  # Empty if columns are missing

    # Merge parameter and attribute pivots, drop the "State" column, and reorder
    if not param_pivot.empty and not attr_pivot.empty:
        merged_df = pd.merge(param_pivot, attr_pivot, on=['number', 'state', 'value'], how='outer').drop(columns=['state'])
    elif not param_pivot.empty:
        merged_df = param_pivot.drop(columns=['state'])
    else:
        merged_df = attr_pivot.drop(columns=['state'])

    # Sorting based on Value (highest or lowest)
    ascending = sort_order == 'Lowest'
    merged_df = merged_df.sort_values(by='value', ascending=ascending)

    # Reorder the columns
    column_order = ['number', 'value', 'acc_train', 'acc_test', 'number_of_clauses'] + \
                   [col for col in merged_df.columns if col not in ['number', 'state', 'value', 'acc_train', 'acc_test', 'number_of_clauses']]
    merged_df = merged_df[column_order]

    limited_df = merged_df.head(max_rows).copy()

    # Format columns as integers with comma separators
    int_columns = ["number_of_clauses", "board_size", "depth", "epochs", "mbf_idx", "n_samples_idx", "open_pos"]
    for col in int_columns:
        if col in limited_df.columns:
            limited_df[col] = pd.to_numeric(limited_df[col], errors='coerce').fillna(0).astype(int).apply(lambda x: f"{x:,}")

    # Format specific columns to six decimal places
    user_attr_columns = ["f1", "precision", "recall", "acc_train", "acc_test"]
    for col in user_attr_columns:
        if col in limited_df.columns:
            limited_df[col] = pd.to_numeric(limited_df[col], errors='coerce').apply(lambda x: f"{x:.6f}" if pd.notna(x) else "")
        
    if 'acc_test' in limited_df.columns:
        limited_df['acc_test'] = pd.to_numeric(limited_df['acc_test'], errors='coerce')

    display_df = limited_df.reset_index(drop=True)
    styled_df = display_df.style.apply(highlight_rows, subset=['acc_test'])

    display(styled_df.hide(axis="index"))


In [6]:
db_handler = DBHandler()

In [7]:
study_id_dropdown, board_size_dropdown = initialize_dropdowns()

sort_order_dropdown = widgets.Dropdown(
    options=['Highest', 'Lowest'],
    description='Sort Order:'
)

row_limit_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=50,
    step=5,
    description='Max Rows:'
)

interactive_plot = widgets.interactive(
    query_top_results,
    study_id=study_id_dropdown,
    board_size=board_size_dropdown,
    sort_order=sort_order_dropdown,
    max_rows=row_limit_slider
)

In [8]:
display(Markdown("### Select Study and Board Size"))
display(interactive_plot)

### Select Study and Board Size

interactive(children=(Dropdown(description='Study ID:', options={'Global_Hex_Study_random': 8, 'Global_Hex_Stu…